In [ ]:

from plyfile import PlyData
import sh_utils
import trimesh
import numpy as np
import plotly.graph_objects as go
%matplotlib qt


import plotly.io as pio
pio.renderers.default='browser'

input_file = "G:/My Drive/Research/gs_data/mov19_2022_03_03/2dgs_output/2385f4c9-0/point_cloud/iteration_500/point_cloud.ply"
vertices = PlyData.read(input_file)["vertex"]


sh = np.column_stack([vertices[key] for key in vertices.data.dtype.names if 'rest' in key or 'dc' in key])
rgb = sh_utils.rgb_from_sh(0,sh,xyz = None,camera_position = None)
xyz = np.column_stack((vertices["x"], vertices["y"], vertices["z"]))
# fig = go.Figure()
# Plotters.scatter3d(fig,xyz[rgb[:,0]<1],rgb[rgb[:,1]<1],3)
# fig.show()

vertices = vertices[(rgb[:,0] < 0.9) & (rgb[:,0] > 0)]
rgb = rgb[(rgb[:,0] < 0.9) & (rgb[:,0] > 0)]


# Create the 3D scatter plot with hover template for color value
fig = go.Figure(data=[go.Scatter3d(
    x=vertices["x"],
    y=vertices["y"],
    z=vertices["z"],
    mode='markers',
    marker=dict(
        size=2,
        color=rgb[:,0],  # Color for each point
        colorscale='Gray',  # Choose a colorscale
        opacity=0.5,  # Use the opacity per point
        colorbar=dict(title="Colorbar")  # Optional colorbar
    ),
    hovertemplate='Color: %{marker.color:.4f}<extra></extra>',  # Show color as float
)])
fig.show()


## 2DGS - Math explained
### Modeling: 
The 2D Gaussian is defined in terms of its local coordinate system, where the $ X$ and $ Y$ axes are scaled according to the Gaussian's shape. The Gaussian is represented in the **world coordinates**, with the **axes** of the Gaussian in world space defined by the tangential vectors $ \mathbf{t_u} $ and $ \mathbf{t_v} $, and its **scaling factors** by $ s_u $ and $ s_v $. These tangential vectors define the directions of the local coordinate axes in the tangent plane (the object FoR).

The **object plane**, which is the tangent plane to the Gaussian in world space, can be described using the plane equation:
$$
P(u, v) = p_k + s_u \mathbf{t_u} u + s_v \mathbf{t_v} v
$$
where $ p_k $ is the center of the Gaussian in world coordinates, and $ u $ and $ v $ represent local coordinates on the tangent plane. This equation expresses the position of any point in the tangent plane in terms of the world coordinates, as modified by the scaling along the tangential axes $ \mathbf{t_u} $ and $ \mathbf{t_v} $.

To transform the local coordinates in the tangent plane (object frame of reference) to world coordinates, we define the following transformation matrix $ H $, which encodes the **scaling**, **rotation**, and **translation** from the object space to the world space:
$$
H = \begin{pmatrix}
s_u \mathbf{t_u} & s_v \mathbf{t_v} & 0 & p_k \\
0 & 0 & 0 & 1
\end{pmatrix}
= \begin{pmatrix}
R & p_k \\
0 & 1
\end{pmatrix}
$$
where:
- $ \mathbf{t_u} = \begin{pmatrix} t_{u_x} \\ t_{u_y} \\ t_{u_z} \end{pmatrix} $ and $ \mathbf{t_v} = \begin{pmatrix} t_{v_x} \\ t_{v_y} \\ t_{v_z} \end{pmatrix} $ are the 3D tangential vectors (defining the axes of the tangent plane in world coordinates),
- $ p_k = \begin{pmatrix} p_{k_x} \\ p_{k_y} \\ p_{k_z} \end{pmatrix} $ is the 3D position of the Gaussian center in world coordinates, and
- $ R $ is the 3x3 rotation matrix that describes the orientation of the tangent plane in world space.

The matrix $ H $ can be interpreted in two parts:
- The first part, $ \begin{pmatrix} s_u \mathbf{t_u} & s_v \mathbf{t_v} & 0 \end{pmatrix} $, represents the scaling and rotation of the axes in the world coordinates. It scales the tangential vectors $ \mathbf{t_u} $ and $ \mathbf{t_v} $ by $ s_u $ and $ s_v $, respectively, and applies any rotational transformation.
- The second part, $ p_k $, represents the **translation** of the Gaussian's center in world coordinates, which shifts the origin of the local tangent plane to the desired world position.

The matrix can also be interpreted in terms of the rotation matrix $ R $ (which describes the orientation of the tangent plane in world space) and the translation vector $ p_k $. The transformation from local coordinates $ (u, v) $ to world coordinates $ (x, y, z) $ is thus achieved through the multiplication of $ H $ by the local coordinate vector:
$$
\begin{pmatrix}
x \\
y \\
z \\
1
\end{pmatrix}
= H \begin{pmatrix}
u \\
v \\
1 \\
1
\end{pmatrix}
$$

This transformation allows us to map a point in the tangent plane (object space) to the corresponding point in the world space, accounting for both the Gaussian's shape (through the scaling $ s_u, s_v $) and its orientation (through the rotation $ R $).

In summary, the matrix $ H $ serves as a **homogeneous transformation** that encapsulates both the geometric properties (scaling and rotation) and the positioning (translation) of the 2D Gaussian in world space.

for every u,v on the tangent plane we can calculate the gaussian power: 
$$ G(u) = \exp\left( -\frac{u^2 + v^2}{2} \right) \tag{6} $$

* $ \mathbf{t_u} $ , $ \mathbf{t_v} , $ $\mathbf{s_u} $ and $\mathbf{s_v}$ are learnable parameters.
* each gaussian is defined by the opacity $\alpha$ and a view dependent color.

### Splatting


The goal of this process is to find the intersection between a ray, originating from a pixel in the image, and the tangent plane of a 2D Gaussian, and then evaluate the power of the Gaussian at the intersection point. Below are the detailed steps.

Step 1: Defining the Image Ray 

We begin by defining the ray from a pixel in the image using two orthogonal planes. The pixel location $(x, y) $ in the image space can be used to define two planes:


* The x-plane (yz): a plane defined by a normal vector $ \mathbf{n}_x = (-1, 0, 0) $ and an offset $ x $. The 4D homogeneous form is $ h_x = (-1, 0, 0, x) $.
* The y-plane (xz): a plane defined by a normal vector $ \mathbf{n}_y = (0, -1, 0) $ and an offset $ y $. The 4D homogeneous form is $ h_y = (0, -1, 0, y) $.


These planes intersect at a ray in 3D space that is represented as the line of intersection between the two planes.

Step 2: Transforming the Planes to the Tangent Plane

Next, we transform the planes from the image space into the local coordinates of the 2D Gaussian primitive (the tangent plane). Using the transformation matrix $ M = (WH)^{-1} $, we apply the inverse transpose to the planes:

$$
h_u = (WH)^\top h_x \\
h_v = (WH)^\top h_y
$$

Where $ W $ is the camera transformation matrix and $ H $ is the transformation matrix from the tangent plane to the world coordinates.

Step 3: Solving for the Intersection

The next step is to solve for the intersection point $(u, v) $ of the ray with the transformed tangent plane. We do this by solving the following system of equations:

$$
h_u \cdot (u, v, 1, 1)^\top = 0 \\
h_v \cdot (u, v, 1, 1)^\top = 0
$$

Expanding these equations gives us:


$$
    h_1^u u + h_2^u v + h_3^u + h_4^u = 0 \\
    h_1^v u + h_2^v v + h_3^v + h_4^v = 0
$$


The solution for $ u $ and $ v $ is obtained by solving this system of equations. This leads to the following expressions for the coordinates $ u(x) $ and $ v(x) $:

$$
u = \frac{h_2^u h_4^v - h_4^u h_2^v}{h_1^u h_2^v - h_2^u h_1^v} \\
v = \frac{h_4^u h_1^v - h_1^u h_4^v}{h_1^u h_2^v - h_2^u h_1^v}
$$

Where $ h_i^u $ and $ h_i^v $ are the homogeneous parameters of the transformed planes.

Step 4: Evaluating the Gaussian at the Intersection

Once we have the coordinates $ (u, v) $, we can evaluate the 2D Gaussian function at the intersection point. The Gaussian function is typically of the form:

$$
G(u, v) = \exp\left( -\frac{u^2 + v^2}{2} \right)
$$

This represents the Gaussian value at the point $ (u, v) $ on the tangent plane.


Once the values of $ u(x) $ and $ v(x) $ are found, we can compute the depth of the intersection point using the following equation:
$$
x = (x_z, y_z, z, z)^\top = W P(u, v) = W H (u, v, 1, 1)^\top
$$
Here, $ W $ is the camera projection matrix, and $ H $ is the transformation matrix from the tangent plane to world coordinates. The last component of the resulting vector $ x = (xz, yz, z, z)^\top $ gives the depth $ z $ of the intersection point (the third element, need to make sure). 